In [ ]:


// a few dependencies for the notebook:
import $ivy.`com.github.pathikrit::better-files:3.9.1`
import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
import $ivy.`com.lihaoyi::scalatags:0.12.0`

import better.files._
import better.files.Dsl._ 

// load historia code

val jarpath = s"/home/bounder/target/scala-2.13/soot_hopper-assembly-0.1.jar"
assert(File(jarpath).exists, "must run jupyter notebook from docker")
interp.load.cp(os.Path(jarpath))

In [ ]:


import edu.colorado.plv.bounder.ir.{CIEnter,CIExit,CBEnter,CBExit}
import edu.colorado.plv.bounder.symbolicexecutor.state.{InitialQuery,ReceiverNonNull}
import edu.colorado.plv.bounder.lifestate.LifeState
import edu.colorado.plv.bounder.lifestate.LifeState.{Not,AbsMsg, LSSpec, LSTrue, Signature,SubClassMatcher}

import edu.colorado.plv.bounder.{Driver,RunConfig, BounderUtil,Action} // Historia utilities
import upickle.default.read
import upickle.default.write
import edu.colorado.plv.bounder.symbolicexecutor.state.{DisallowedCallin,NamedPureVar,TopVal, InitialQueryWithStackTrace, Reachable, NullVal}
import edu.colorado.plv.bounder.lifestate.SAsyncTask
import edu.colorado.plv.bounder.PickleSpec
import edu.colorado.plv.bounder.lifestate.{AllMatchers,SJavaThreading,FragmentGetActivityNullSpec, LifeState, LifecycleSpec, RxJavaSpec, SAsyncTask, SDialog, SpecSignatures, SpecSpace, ViewSpec}
import edu.colorado.plv.bounder.lifestate.LifeState.{AbsMsg, LSSpec, LSTrue,LSFalse,And, Signature,SubClassMatcher}
import edu.colorado.plv.bounder.symbolicexecutor.{PreciseApproxMode,LimitAppRecursionDropStatePolicy,LimitMsgCountDropStatePolicy,DumpTraceAtLocationPolicy,LimitLocationVisitDropStatePolicy, 
                                                  LimitCallStringDropStatePolicy,LimitCallbackRepetitionDropStatePolicy, LimitMaterializedFieldsDropStatePolicy,DropMaterializedFieldsPolicy}
import edu.colorado.plv.bounder.symbolicexecutor.Z3TimeoutBehavior

def getIsDocker():Boolean = {
    val res = BounderUtil.runCmdStdout("whoami")
    res.trim == "root"
}

val isDocker = getIsDocker()
// val isDocker = false //TODO: overridden

val historiaDir = if(isDocker) "/home/bounder" else "/Users/shawnmeier/Documents/source/historia/Historia/"

// define a function to call the JAR implementation of Historia with a configuration
// If changes are made to Historia, run "sbt compile" in the /home/implementation directory to regenerate the Historia JAR

def runHistoriaWithSpec(configPath:File, printThenDone:Boolean = false, outputMode:String = "MEM"):String = {
    val javaMemLimit=20 // Gb Note that this only limits JVM not JNI which can go significantly higher
    val historiaJar = jarpath
    val apkRootDir = "/Users/shawnmeier/Documents/data/historia_generalizability"
    val outDir = configPath.parent.toString
    val config = read[RunConfig](configPath.contentAsString)
    val outSubdir = config.outFolder.get.replace("${baseDirOut}",outDir)
    val cmd = s"java -Xmx${javaMemLimit}G -jar ${historiaJar} -m verify -c ${configPath} -b ${apkRootDir} -u ${outDir} -o ${outputMode} --debug"
    
    if(printThenDone){
        println(cmd)
        cmd
    }else{
        //BounderUtil.runCmdStdout(cmd, Some("/Users/shawnmeier/software/z3/build"))
        BounderUtil.runCmdFileOut(cmd, configPath.parent).toString
    }
}

// def printOutput(

def runAndPrint(configPath:File, allSpecs:Iterable[LSSpec], printThenDone:Boolean = false, outputMode:String = "MEM"):String = {
    val res = runHistoriaWithSpec(configPath,printThenDone, outputMode)
    println("run result")
    println(res)   
    println("specified messages")
    val msgSigs = allSpecs.flatMap(spec =>
                Set(spec.target) ++ spec.pred.allMsg).map(msg => msg.identitySignature)
    println(msgSigs)
    println(msgSigs.size)
    res
}

val v = NamedPureVar("v")
val a = NamedPureVar("a")
val f = NamedPureVar("f")
val s = NamedPureVar("s")
val l = NamedPureVar("l")
val t = NamedPureVar("t")
val r = NamedPureVar("r")

Scalability Experiment
======================

Connectbot GeneratePubkeyActivity$5 dialog dismiss

In [ ]:
// The method signature unambiguously identifies the method in the application
// note: timing out a lot, giving up for now 2/14/24

// idea: drop states where we have more than one instance of object of some type
// discussion in paper: we can't say we find "shortest history"
// Should justify heuristics and add to threats to validity
// is there a way to evaluate the improvement before completion?

// can we add "unsound" constraints to fwk model? e.g.  cb r.run() -[]-> HN r.run()
// partial order reduction -- before exploring all paths, show that ab and ba is same -- can we fix order of fwk model?

val dataDir = if(isDocker) "/home/reach_24_data/" else "/Users/shawnmeier/Documents/data/reach_24_data/"
val notebooksDir = if(isDocker) "/home/notebooks/" else s"${historiaDir}/notebooks/"
val v = NamedPureVar("v")
val a = NamedPureVar("a")

def runCBGenPub(){
    // val methodSignature = LifeState.Signature(
    //     "com.example.row1antennapodrxjava.ui.main.PlayerFragment",
    //           "void call(java.lang.Object)")


    val initialQuery = DisallowedCallin(className = "org.connectbot.GeneratePubkeyActivity$5",
                                        methodName = "void run()",
                                        s=SDialog.disallowDismiss)


    val inputApk = s"${dataDir}connectbot/app/build/outputs/apk/oss/debug/app-oss-debug.apk"
    val outputDir = File(s"/home/notebooks/reachExpLarge/Connectbot/GeneratePubkeyActivity_5_run")
    val configOutputDir = File(s"${notebooksDir}/reachExpLarge/Connectbot/GeneratePubkeyActivity_5_run")
    // val activityMatchers = AllMatchers.iContaining("Activity")
    // println(activityMatchers)

    val onTouchWhileActive: LSSpec = LSSpec(v :: Nil, a :: Nil, LifecycleSpec.viewAttached,
    // And( LifecycleSpec.viewAttached, Or(LifecycleSpec.resumed,
    //   AbsMsg(CIExit, SpecSignatures.Activity_finish, TopVal :: a :: Nil))),
    ViewSpec.onTouchEvent)

    val onlyOnce:Set[LSSpec] = Set(
LSSpec(r::Nil,Nil, Not(SJavaThreading.runnableI),SJavaThreading.runnableI),
LSSpec(a::Nil,Nil, Not(SpecSignatures.Activity_onPause_exit.copy(lsVars=Nil)),SpecSignatures.Activity_onPause_exit),
LSSpec(a::Nil,Nil, Not(SpecSignatures.Activity_onResume_exit.copy(lsVars=Nil)),SpecSignatures.Activity_onResume_exit),
        )
    
    val cfg =  RunConfig(apkPath = inputApk.toString, 
                         timeLimit=Int.MaxValue,
                         // limit = 600, //TODO: limits number of steps for testing
                    outFolder = Some(configOutputDir.toString),
                    initialQuery = List(initialQuery), truncateOut=false,
                    specSet = PickleSpec(Set(onTouchWhileActive) ++ onlyOnce,
                                             Set(SDialog.disallowDismiss), 
                                             Set(SpecSignatures.Activity_onCreate_entry,SJavaThreading.runnableI)
                                        ),
                    componentFilter = 
                         Some(Seq("org.connectbot.GeneratePubkeyActivity.*",
                                  "org.connectbot.util.EntropyView.*", 
                                  "org.connectbot.util.EntropyDialog.*",
                                  "org.connectbot.util.PubkeyDatabase.*")),
                    approxMode = PreciseApproxMode(true,  List(LimitLocationVisitDropStatePolicy(3), LimitCallStringDropStatePolicy(15))),
                    z3TimeoutBehavior = Some(Z3TimeoutBehavior().copy(
                            subsumeTryTimeLimit=List(400000), 
                                                                      // excludesInitTimeout = 1000000, 
                                                                      z3InstanceLimit=4))
                        )
    // val action=Action(config=cfg, outputMode="DB")
    // action.runCmdFork(jarpath)


    val cfgPath = (outputDir / "cfg.json")
    val configCfgPath = configOutputDir / "cfg.json"
    cfgPath.overwrite(write(cfg))
    
    val allSpecs = Set(SDialog.disallowDismiss)

    val justPrintCommand = false
    println(runAndPrint(cfgPath, allSpecs,justPrintCommand).replace(cfgPath.toString,configCfgPath.toString))

    if(!justPrintCommand)
        println(read[Driver.LocResult]((outputDir / "result_0.txt").contentAsString).witnessExplanation)
}
runCBGenPub()



In [ ]:
println(read[Driver.LocResult]((File(s"/home/notebooks/reachExpLarge/Connectbot/GeneratePubkeyActivity_5_run") / "result_0.txt").contentAsString))

In [ ]:
// measure for how "useful" this trace is?
// show app is fixed after? but doesn't show we used trace
// 1. can I generate witness with wistoria
//   [ ] come up with multiple witnesses prefereably from "in the wild" issues
//   [ ] 
// 2. can I use trace to witness crash?
// 3. can I fix the app (hopefully with trace help?)
// 4. could we get response to trace that drives interaction? Even without cras


Connectbot unregisterMenuListeners
https://github.com/connectbot/connectbot/issues/1229

In [ ]:
val dataDir = if(isDocker) "/home/reach_24_data/" else "/Users/shawnmeier/Documents/data/reach_24_data/"
val notebooksDir = if(isDocker) "/home/notebooks/" else s"${historiaDir}/notebooks/"
val v = NamedPureVar("v")
val a = NamedPureVar("a")
val m = NamedPureVar("m")

def runCBConsoleAct(){



    val methodSignature = LifeState.Signature("org.connectbot.ConsoleActivity",
          "void unregisterMenuListeners()")

val internalQuery = ReceiverNonNull(
        methodSignature,
        385, // line number in source code file
        Some(".*setOnMenu.*")
    )
    val trace = """at org.connectbot.ConsoleActivity.unregisterMenuListeners(ConsoleActivity.java:414)
| at org.connectbot.ConsoleActivity.closeBridge(ConsoleActivity.java:409)
| at org.connectbot.ConsoleActivity.onDisconnected(ConsoleActivity.java:217)
| at org.connectbot.service.TerminalManager.onDisconnected(TerminalManager.java:344)
| at org.connectbot.service.TerminalBridge$7.run(TerminalBridge.java:510)
|""".stripMargin

    val initialQuery = InitialQueryWithStackTrace.fromStackTrace(trace, internalQuery)


    val inputApk = s"${dataDir}connectbot/app/build/outputs/apk/oss/debug/app-oss-debug.apk"
    val outputDir = File(s"/home/notebooks/reachExpLarge/Connectbot/ConsoleActivity_setMenuClick_NPE")
    val configOutputDir = File(s"${notebooksDir}/reachExpLarge/Connectbot/ConsoleActivity_setMenuClick_NPE")
    // val activityMatchers = AllMatchers.iContaining("Activity")
    // val onCreateOptionsMenu = AbsMsg(CIExit, 
    //                                  SubClassMatcher(SpecSignatures.Activity,"boolean onCreateOptionsMenu\\(.*","Activity_onCreateOptionsMenu"),
    //                                  TopVal::
    // val menuAddNoNull = LSSpec(m::Nil, Nil, And(onCreateOptionsMenu, SpecSignatures.finish), 
    //                            AbsMsg(CIExit, SubClassMatcher("android.view.Menu","android.view.MenuItem add\\(.*","Menu_add"),NullVal::Nil))
    val menuAddNoNull = LSSpec(m::Nil, Nil, LSFalse, 
                               AbsMsg(CIExit, SubClassMatcher("android.view.Menu","android.view.MenuItem add\\(.*","Menu_add"),NullVal::Nil))
    val runMsg = SJavaThreading.runnableI
    val handlerPostMsg = AbsMsg(CIExit, SubClassMatcher("android.os.Handler" ,"boolean post\\(.*","Handler_post"), TopVal::TopVal::r::Nil)
    val runAfterPost = LSSpec(r::Nil, Nil, LSTrue, runMsg) //TODO: strengthen
    
    val cfg =  RunConfig(apkPath = inputApk.toString, 
                         timeLimit=Int.MaxValue,
                    outFolder = Some(configOutputDir.toString),
                    initialQuery = List(initialQuery), truncateOut=false,
                    specSet = PickleSpec(Set(menuAddNoNull), //runAfterPost
                                             Set(), 
                                             Set()//SpecSignatures.Activity_onCreate_entry,SJavaThreading.runnableI)
                                        ), //,
                                             // SpecSignatures.Activity_findView_exit)), //, ViewSpec.onTouchEvent)),
                    componentFilter = Some(Seq("org.connectbot.ConsoleActivity.*",
                                               "org.connectbot.HostListActivity.*", // onCreateViewHolder
                                               "org.connectbot.service.*TerminalManager.*",
                                               "org.connectbot.service.*TerminalBridge.*", // finishConnection onNewIntent
                                               "org.connectbot.PortForwardListActivity.*",
                                               "org.connectbot.TerminalView.*",
                                               "org.connectbot.transport.Local.*",
                                               "org.connectbot.transport.SSH.*",
                                               "org.connectbot.transport.AbsTransport.*",
                                               //"org.connectbot.service.Relay.*", //  .*Relay for run
                                               // "method:.* setBridge\\(.*",
                                               // "method:.* instantiateItem\\(.*",
                                               // "method:.* connect\\(.*",
                                               // "method:.* startConnection\\(.*",
                                               // "method:.* openConnection\\(.*",
                                               // "method:.* reconnectPending\\(.*",
                                               // "method:.* requestReconnect\\(.*",
                                               // "method:.* onConnectivityRestored\\(.*",
                                               // "method:.* connectionLost\\(.*",
                                               // "method:.* read\\(.*",
                                               // "method:.* run\\(.*",
                                               // "method:.* onDisconnected\\(.*",
                                               // "method:.* closeBridge\\(.*",
                                               // "method:.* unregisterMenuListeners\\(.*",
                                               // "method:.* triggerDisconnectListener\\(.*",
                                               // "method:.* dispatchDisconnect\\(.*",
                                               // "method:.* onCreateOptionsMenu\\(.*",
                                               // "method:.* onMenuItemClick\\(.*",
                                               // "method:.* onCreateContextMenu\\(.*",
                                               // "method:.* onCreate\\(.*",
                                               // "method:.* isAwaitingClose\\(.*",
                                               // "method:.* <init>\\(.*",
                                               // "method:.* onServiceConnected\\(.*",
                                               // "method:.* setOnDisconnectedListener\\(.*",
                                               // "method:.* openConnection\\(.*",
                                               // "method:.* triggerDisconnectListener\\(.*",
                                               // "method:.* set.*\\(.*",
                                               // "method:.* onConnected\\(.*",
                                               // "method:.* isDisconnected\\(.*",
                                               )),
                    approxMode = PreciseApproxMode(true, 
                                                   List(LimitLocationVisitDropStatePolicy(4), 
                                                     // LimitCallStringDropStatePolicy(5),
                                                     LimitMaterializedFieldsDropStatePolicy(Map("*" -> 3)),
                                                        // DropMaterializedFieldsPolicy("disconnected"), //TODO: this seems to make wit?
                                                     // DropMaterializedFieldsPolicy("host"),
                                                        // DropMaterializedFieldsPolicy("awaitingClose"),
                                                        // DropMaterializedFieldsPolicy("portForward"),
                                                        // DropMaterializedFieldsPolicy("quickDisconnect"),
                                                        // DropMaterializedFieldsPolicy("stayConnected"),
                                                        // DropMaterializedFieldsPolicy("BEAN_NAME"),
                                                        // DropMaterializedFieldsPolicy("DEFAULT_FONT_SIZE"),
                                                        // DropMaterializedFieldsPolicy("id"),
                                                        // DropMaterializedFieldsPolicy("nickname"),
                                                        // DropMaterializedFieldsPolicy("username"),
                                                        // DropMaterializedFieldsPolicy("hostname"),
                                                        // DropMaterializedFieldsPolicy("port"),
                                                        // DropMaterializedFieldsPolicy("protocol"),
                                                        // DropMaterializedFieldsPolicy("lastConnect"),
                                                        // DropMaterializedFieldsPolicy("color"),
                                                        // DropMaterializedFieldsPolicy("useKeys"),
                                                        // DropMaterializedFieldsPolicy("useAuthAgent"),
                                                        // DropMaterializedFieldsPolicy("postLogin"),
                                                        // DropMaterializedFieldsPolicy("pubkeyId"),
                                                        // DropMaterializedFieldsPolicy("wantSession"),
                                                        // DropMaterializedFieldsPolicy("delKey"),
                                                        // DropMaterializedFieldsPolicy("fontSize"),
                                                        // DropMaterializedFieldsPolicy("compression"),
                                                        // DropMaterializedFieldsPolicy("encoding"),
                                                        // DropMaterializedFieldsPolicy("stayConnected"),
                                                        // DropMaterializedFieldsPolicy("quickDisconnect"),

                                                        //TODO: this is hacky and hard to justify, figure out how to remove???
                                                        DumpTraceAtLocationPolicy(None,None,outputDir.toString, 
                                                                                  dumpAtCallbacks = true),
                                                        LimitCallbackRepetitionDropStatePolicy(1),
                                                       )), 
                                                   // ++
                                                     // initialQuery.trace.map{c => 
                                                     //   DumpTraceAtLocationPolicy(Some(c), None, 
                                                     //     outputDir.toString, dumpAtCallbacks = true)}
                                                     // ),
                    z3TimeoutBehavior = Some(Z3TimeoutBehavior().copy(
                            subsumeTryTimeLimit=List(200000), 
                                                   // excludesInitTimeout = 1000000, 
                                                   z3InstanceLimit=16))
                        )


    if(!outputDir.isDirectory){
        mkdir(outputDir)
    }
    val cfgPath = (outputDir / "cfg.json")
    val configCfgPath = configOutputDir / "cfg.json"
    cfgPath.overwrite(write(cfg))
    
    val allSpecs = Set(SDialog.disallowDismiss)

    val justPrintCommand = false
    println(runAndPrint(cfgPath, allSpecs,justPrintCommand).replace(cfgPath.toString,configCfgPath.toString))

    if(!justPrintCommand){
        val result = read[Driver.LocResult]((outputDir / "result_0.txt").contentAsString)
        println(result.witnessExplanation)
        println(result.resultSummary)
    }
}
runCBConsoleAct()